# Tarea 4
0226594 || Sara Carolina Gómez Delgado

### Utils

In [3]:
import matplotlib
from sklearn.ensemble import RandomForestClassifier
import pandas
from bs4 import BeautifulSoup
import numpy as np
import scipy
import nltk 
from nltk.tokenize import TweetTokenizer
from itertools import islice
import random
import mpmath
from collections import defaultdict
from tqdm import tqdm

In [4]:
def get_texts_from_file(path_corpus, path_truth):
    tr_txt = []
    tr_y = []
    with open(path_corpus, "r",encoding="utf8") as f_corpus, open(path_truth, "r",encoding="utf8") as f_truth:
        for tuit in f_corpus:
            tr_txt += [tuit]
        for label in f_truth:
            tr_y += [label] 
    return tr_txt, tr_y

---
<h1 style="text-align: center;">Parte 1</h1>

<h4 style="text-align: center;"><i>"Modelos de Lenguaje y Evaluación"</i></h4>

---

#### 1.1 Preprocesamiento

_<kbd>Instrucción</kbd>_

Preprocese todos los tuits de agresividad (positivos y negativos) según su intuición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, etc.). Agregue tokens especiales de < s > y </ s > según usted considere (e.g., al
inicio y final de cada tuit). Defina su vocabulario y enmascare con < unk > toda palabra
que no esté en su vocabulario.


_<kbd>Comentario</kbd>_

En esta sección se preprocesaron los tweets y se generó un corpus donde sólo se consideraron palabras (no emojis, no signos de puntuación) y se eligió un vocabulario tomando las palabras únicas en el training set, después se ordenaron según su frecuencia (mayor a menor) y se tomaron sólo las palabras cuya frecuencia fue mayor a 50 para formar el vocabulario final. Me parece importante resaltar, como principal conclusión, la importancia de descartar palabras que aparecen poco, ya que pueden llegar a afectar fuertemente al intentar aplicar un modelo de lenguaje.

_Leer tweets_

In [5]:
tr_txt, tr_y = get_texts_from_file("./mex_train.txt", "./mex_train_labels.txt")

In [6]:
tokenizer = TweetTokenizer()

_Pre-procesar tweets_

In [7]:
# <s> at the beggining of a tweet and </s> at the end of this tweet
corpus_palabras = []
for doc in tr_txt:
    x = "<s>" + doc + "</s>"
    corpus_palabras += tokenizer.tokenize(x) 

In [8]:
processed = []
for word in corpus_palabras:
    if(np.char.isalpha(word) or word == "<s>" or word == "</s>"):
            processed.append(word)
print(processed[:10])

['<s>', 'lo', 'peor', 'de', 'todo', 'es', 'que', 'no', 'me', 'dan']


In [9]:
fdist = nltk.FreqDist(processed) # frecuencia de cada palabra
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict] #lista de pares ordenada (más frecuente a menos)
    aux.sort()
    aux.reverse()
    return aux #regresa el objeto ordenado en reversa (más frecuentes a menos frecuentes)

V = sortFreqDict(fdist)

In [10]:
# if freq < 50, ignore it (don't let it be part of vocab)
vocab = []
for item in V:
    if item[0] > 1:
        vocab.append(item[1])
vocab[:10]
print(len(vocab))


4581


In [11]:
for i,word in enumerate(processed):
    if word not in vocab:
        processed[i] = "<unk>"
print(processed[:10])

['<s>', 'lo', 'peor', 'de', 'todo', 'es', 'que', 'no', 'me', 'dan']


#### 1.2 Entrenamiento (unigrama, bigrama y trigrama)

_<kbd>Instrucción</kbd>_

Entrene tres modelos de lenguaje sobre todos los tuis: $P_{unigramas}(w^n_1), P_{bigramas}(w^n_1), P{trigramas}(w^n_1).$ Para cada uno proporcione  una interfaz (función) sencilla para $P{n-grama}(w^n_1)$ y $Pn-grama(w^n_1 | w^{n-1}_{n-N+1})$. Los modelos modelos deben tener una estrategia común para lidiar con
secuencias no vistas. Puede optar por un suavizamiento Laplace o un Good-Turing
discounting. Muestre un par de ejemplos de como funciona, al menos uno con una
palabra fuera del vocabulario.


_<kbd>Comentario</kbd>_

En esta sección creé dos funciones aplicando en ambas Laplace Smoothing (Add-One) para manejar el caso de secuencias no vistas.
1) Función que devuelve la probabilidad de que suceda una palabra dada un contexto.
2) Función que devuelve la probabilidad de que suceda una oración.

Como conclusion, me llamó mucho la atención el hecho de que cuando se encuentra el modelo con una palabra que no ha visto antes, no la vuelve cero (por al add-one de Laplace), si no que su probabilidad se vuelve muy pequeña.

_N-grams_

In [12]:
def unigram(words, context, vocab_size):
    word = words[-1]
    word_freq = context.count(word)
    context_size = len(context)
    return (word_freq+1)/(context_size+vocab_size)

def bigram(words, context, vocab_size):
    a = words[-2]
    b = words[-1]
    a_freq = context.count(a)
    bigram_freq = 0
    for i in range(len(context)-1):
        if context[i]==a and context[i+1]==b:
            bigram_freq += 1
    return (bigram_freq+1)/(a_freq+vocab_size)

def trigram(words, context, vocab_size):
    a = words[-3]
    b = words[-2]
    c = words[-1]
    count_trigram = 0
    count_bigram = 0
    for i in range(len(context)-2):
        # trigrama
        if context[i:i+3] == [a,b,c]:
            count_trigram += 1
        # bigrama
        if context[i:i+2] == [b,c]:
            count_bigram += 1
    return (count_trigram+1)/(count_bigram + vocab_size)


_Probabilidad de que ocurra una palabra dada el contexto_

In [13]:
def prob_word(words, context, vocab_size): # (list)
    assert isinstance(words, list)
    assert isinstance(context, list)
    assert len(context) > 0
    if len(words) == 1: 
        print("unigrama")
        prob = unigram(words, context, vocab_size)
        assert prob != 0
        return prob 
    elif len(words) == 2:
        print("bigrama")
        prob = bigram(words, context, vocab_size)
        assert prob != 0
        return prob 
    elif len(words) == 3: 
        print("trigrama")
        prob = trigram(words, context, vocab_size)
        assert prob != 0
        return prob 

# unigrama
result = prob_word(["a"], processed, len(processed))
print("\t{:.10f}".format(result))

# bigrama
result = prob_word(["lo", "que"], processed, len(processed)) 
print("\t{:.10f}".format(result))

# trigrama
result = prob_word(["si", "no", "fuera"], processed, len(processed))
print("\t{:.10f}".format(result))

# These words don't belong to the vocabulary
result = prob_word(["hello", "dude"], processed, len(processed))
print("\t{:.10f}".format(result))

unigrama
	0.0133005624
bigrama
	0.0013767184
trigrama
	0.0000404555
bigrama
	0.0000101145


_Probabilidad de que ocurra una oración_

In [14]:
def sentence_prob(n, sentence, corpus):
    assert isinstance(n, int)
    assert isinstance(sentence, list)
    assert n <= len(sentence)
    assert isinstance(corpus, list)

    if n == 1:
        uni_prob = 1
        for word in sentence:
            uni = unigram(word, sentence, len(sentence))
            uni_prob *= uni
        assert uni_prob != 0
        return uni_prob
    elif n == 2:
        bi_prob = 1
        for i in range(0, len(sentence)- 2):
            bi = bigram(sentence[i:i+2], sentence, len(sentence))
            bi_prob *= bi
        return bi_prob
    elif n == 3:
        tri_prob = 1
        for i in range(0, len(sentence) - 3):
            tri = trigram(sentence[i:i+3], sentence, len(sentence))
            tri_prob *= tri
        return tri_prob
ngram = 3
result = sentence_prob(ngram, ['de', 'todo', 'lo', "que", "te", "digo", "nada", "me", "haces", "caso"], processed)
if ngram == 1: print("unigram")
elif ngram == 2: print("bigram")
elif ngram == 3: print("trigram")
else: print("n-gram")
print("\t{:.10f}".format(result))


trigram
	0.0000065684


#### 1.3 Construcción de Modelo Interpolado

_<kbd>Instrucción</kbd>_

Construya un modelo interpolado con valores $\lambda$ fijos:

$$\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1P(w_n|w_{n-2}w_{n-1}) + \lambda_2P(w_n|w_{n-1}) + \lambda_3P(w_n)$$

Para ello experimente con el modelo en particiones estratificadas de 80%, 10% y 10% para
entrenar (train), ajuste de parámetros (val) y prueba (test) respectivamente. Muestre como
bajan o suben las perplejidades en validación, finalmente pruebe una vez en test. Para esto puede explorar algunos valores ⃗λ y elija el mejor, i.e., [1/3, 1/3, 1/3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1]
y [.1, .4, .5].

_<kbd>Comentario</kbd>_

En esta sección dividí primero mis datos en 80% train, 10% validation y 10% test. Después, creé el modelo interpolado donde $\lambda_1P(w_n|w_{n-2}w_{n-1})$ representa un trigrama multiplicado por un valor lambda_1, \lambda_2P(w_n|w_{n-1}) representa un bigrama multiplicado por un valor de lambda_2 y \lambda_3P(w_n) representa un unigrama multiplicado por un valor de lambda_3. Los resultados son sumados. Sumé estos valores y finalmente experimenté con los diferentes valores de lambda para comparar sus perplejidades y quedarme con el valor más pequeño.

_Data Partition (80% (train), 10% validation, 10% test)_

In [15]:
# Shuffle data
rand_data = processed.copy()
random.shuffle(rand_data)

# Divide 80% Train, 20% Test
n = len(rand_data) 
n_train = int(n * 0.8)  # entrenamiento (80%)
n_test = n - n_train  # prueba (20%)
train = rand_data[:n_train]
to_divide = rand_data[n_train:]

# Divide that 20% in two parts (10% Validation, 10% Test)
n_test = len(to_divide) 
n_val = n_test//2  # Validation (50%)
n_test = n_test - n_val  # Test(50%)
validation = to_divide[:n_val]
test = to_divide[n_val:]

print("Train: ",(len(train)*100)/len(processed))
print("Validation: ",(len(validation)*100)/len(processed))
print("Test: ",(len(test)*100)/len(processed))

Train:  79.99959542015617
Validation:  10.000202289921916
Test:  10.000202289921916


In [16]:
def interpolated_model(words, context, vocab_size, lambdas):
    return lambdas[0]*unigram(words, context, vocab_size) + lambdas[1]*bigram(words, context, vocab_size) + lambdas[2]*trigram(words, context, vocab_size)

In [17]:
def perplexity(lambdas,context,N): 
    # Unigram(...)*lambda[0] + Bigram(...)*lambda[1] + Trigram(...)*lambda[2] +
    context = ["<s>"]+context+["</s>"]
    result = 1
    for i in range(len(context)):
        u = unigram(context[i], context, N)*lambdas[0]

        if i < 1:  b = bigram("<s>" + context[i], context, N)*lambdas[1]
        else: b = bigram(context[i - 1: i + 1], context, N)*lambdas[1]
        
        if i < 2:  t = trigram("<s>" + "<s>" +context[i], context, N)*lambdas[2]
        else: t = trigram(context[i-2: i+1], context, N)*lambdas[2]
        prob = u + b + t
        inv = 1/prob
        result *= mpmath.root(inv, N+1)
    print(result)
    return result

_Find Best Lambda_

In [18]:
def best_lambda(lambdas, N, context):
    assert len(lambdas) >= 1
    assert len(context) >= 3
    best_lam = lambdas[0]
    best_perplexity = 10000
    for l in lambdas:
        p = perplexity(l,context,N)
        if(p < best_perplexity):
            best_perplexity = p
            best_lam = [l[0], l[1], l[2]]
    return best_lam, best_perplexity

_Validation_

In [19]:
lambdas = [[1/3, 1/3, 1/3], [.4, .4, .2], [.2, .4, .4], [.5, .4, .1], [.1, .4, .5]]

res = best_lambda(lambdas, len(validation), validation)
validation_best_lambda = res[0]
validation_best_perplexity = res[1]
print("best lambda and its perplexity: ",validation_best_lambda, validation_best_perplexity)

1857.12498840599
1736.41139145149
2048.26062839668
1654.40733733991
2395.50120471283
best lambda and its perplexity:  [0.5, 0.4, 0.1] 1654.40733733991


_Test_

In [20]:
lambdas = [[1/3, 1/3, 1/3], [.4, .4, .2], [.2, .4, .4], [.5, .4, .1], [.1, .4, .5]]

res = best_lambda(lambdas, len(test), test)
test_best_lambda = res[0]
test_best_perplexity = res[1]
print("best lambda and its perplexity: ", test_best_lambda, test_best_perplexity)

1949.53844704973
1836.98453146326
2106.22561520533
1772.5647626856
2420.95307783619
best lambda and its perplexity:  [0.5, 0.4, 0.1] 1772.5647626856


_Show Best Perplexity_

In [24]:
global_best_lambda = [1,1,1]
global_best_perplexity = 100000
if validation_best_perplexity < test_best_perplexity:
    print("Best Perplexity (validation): ", validation_best_perplexity)
    print("Lambdas: ", validation_best_lambda)
    global_best_lambda = validation_best_lambda
    global_best_perplexity = validation_best_perplexity
else:
    print("Best Perplexity (test): ", test_best_perplexity)
    print("Lambdas: ", test_best_lambda)
    global_best_lambda = test_best_lambda
    global_best_perplexity = test_best_perplexity

Best Perplexity (validation):  1654.40733733991
Lambdas:  [0.5, 0.4, 0.1]


---
<h1 style="text-align: center;">Parte 2</h1>

<h4 style="text-align: center;"><i>"Generación de Texto"</i></h4>

---

Para esta parte reentrenará su modelo de lenguaje interpolado para aprender los valores λ:

$$\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1P(w_n|w_{n-2}w_{n-1}) + \lambda_2P(w_n|w_{n-1}) + \lambda_3P(w_n)$$

#### 2.1 Función "tuitear"

_<kbd>Instrucción</kbd>_

Haga una función "tuitear" con base en su modelo de lenguaje P̂ del último punto.
El modelo deberá poder parar automáticamente cuando genere el símbolo de terminación
de tuit al final (e.g., "< /s >"), o 50 palabras. Proponga algo para que en los últimos tokens
sea más probable generar el token "</ s >". Muestre al menos cinco ejemplos.


_<kbd>Comentario</kbd>_



In [46]:
# print(len(processed))
# def generate_word(corpus):
#    probs = []
#    for i in range(0, len(corpus)):
#       p = interpolated_model(["<s>","<s>",corpus[i]],corpus, len(corpus), global_best_lambda)
#       probs.append(p)
#       print(p)
#    return np.random.choice(corpus, p=probs/np.sum(probs))
def generate_word(corpus):
   trigrams = [(corpus[i-2], corpus[i-1], corpus[i]) for i in range(2, len(corpus))]
   probs = interpolated_model(trigrams, corpus, len(corpus), global_best_lambda)
   probs = np.array(probs)
   if np.sum(probs) == 0:
      return np.random.choice(corpus[2:])
   else:
      probs /= np.sum(probs)
      return np.random.choice(corpus[2:], p=probs)

print("WORD: ", generate_word(processed))

['peor', 'de', 'todo', 'es', 'que', 'no', 'me', 'dan', 'por', 'un', 'tiempo', 'y', 'luego', 'vuelven', 'estoy', 'hasta', 'la', 'verga', 'de', '<unk>', '</s>', '<s>', 'a', 'la', 'vga', 'no', 'seas', 'mamón', 'putos', 'minutos', 'después', 'me', 'dices', 'que', 'apenas', 'sales', 'no', 'me', 'querías', 'avisar', 'en', 'horas', '</s>', '<s>', 'considero', 'que', 'lo', 'más', '<unk>', 'seria', 'que', 'lo', '<unk>', 'a', 'unos', 'vergazos', 'mi', '<unk>', '<unk>', 'la', 'madre', 'a', 'ese', 'pinchi', 'joto', '</s>', '<s>', 'el', 'marica', 'de', 'mi', 'ex', 'me', 'tiene', '<unk>', 'de', 'todo', 'así', 'uno', 'no', 'puede', 'admirar', 'la', 'belleza', 'de', 'su', '<unk>', '</s>', '<s>', 'mujer', '<unk>', 'pinche', 'amlo', '<unk>', 'esta', 'que', 'se', 'pela', 'la', 'loca', '</s>', '<s>', 'putos', 'no', 'tienen', 'madre', '<unk>', '<unk>', 'ojetes', 'como', 'es', 'posible', 'mejor', '<unk>', '</s>', '<s>', 'ustedes', 'si', '<unk>', 'andar', 'de', '<unk>', 'pero', '<unk>', 'y', 'seamos', 'nosot

TypeError: len() of unsized object

#### 2.2 Entrenar modelo de lenguaje AMLO

_<kbd>Instrucción</kbd>_

Use la intuición que ha ganado en esta tarea y los datos de las mañaneras para
entrenar un modelo de lenguaje AMLO. Haga una un función "dar_conferencia()". Generé
un discurso de 300 palabras y detenga al modelo de forma abrupta.

In [23]:
def dar_conferencia():
    pass

#### 2.3 Experimentando con dos frases

_<kbd>Instrucción</kbd>_

Calcule el estimado de cada uno sus modelos de lenguaje (el de tuits y el de amlo)
para las frases: "sino gano me voy a la chingada", "ya se va a acabar la corrupción".

#### 2.4 Permutaciones

_<kbd>Instrucción</kbd>_

Para cada oración del punto anterior, haga todas las permutaciones posibles.
Calcule su probabilidad a cada nueva frase y muestre el top 3 mas probable y el top 3
menos probable (para ambos modelos de lenguaje). Proponga una frase más y haga lo
mismo.